In [307]:
%pip install pandas matplotlib
%pip install scikit-learn
%pip install xgboost

/Users/joelsng/Documents/GitHub/levelsliving-IMS/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/joelsng/Documents/GitHub/levelsliving-IMS/.venv/lib/python3.12/site-packages/pip/__main__.py", line 22, in <module>
    from pip._internal.cli.main import main as _main
  File "/Users/joelsng/Documents/GitHub/levelsliving-IMS/.venv/lib/python3.12/site-packages/pip/_internal/__init__.py", line 3, in <module>
    from pip._internal.utils import _log
ModuleNotFoundError: No module named 'pip._internal.utils'
Note: you may need to restart the kernel to use updated packages.
  Using cached xgboost-3.0.5-py3-none-macosx_12_0_arm64.whl.metadata (2.1 kB)
  Using cached numpy-2.3.4-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached scipy-1.16.2-cp312-cp312-macosx_14_0_arm64.whl

In [428]:
import pandas as pd
import matplotlib as plt
from datetime import timedelta

In [429]:
df = pd.read_csv("sampleData/master_orders.csv")
df["Order Date"] = pd.to_datetime(df["Order Date"], utc=True)
df = df.sort_values("Order Date")

In [430]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 17684 entries, 17563 to 1685
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   Order Date     17506 non-null  datetime64[ns, UTC]
 1   SKU            16507 non-null  object             
 2   Item           17683 non-null  object             
 3   Variant        14338 non-null  object             
 4   Quantity       17663 non-null  float64            
 5   Delivery Date  11825 non-null  object             
 6   Delivery Time  11573 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(1), object(5)
memory usage: 1.1+ MB
None


# Aggregate based on monthly

In [431]:
monthly = (
    df
    .set_index("Order Date")
    .groupby(["SKU", pd.Grouper(freq="ME")])["Quantity"]
    .sum()
    .reset_index()
    .rename(columns={"Order Date": "Month"})
    .sort_values(["SKU", "Month"])
)

# Feature Engineering

In [432]:
monthly["month_num"] = monthly["Month"].dt.month

monthly["year"] = monthly["Month"].dt.year

monthly["sku_id"] = monthly["SKU"].astype("category").cat.codes

# demand lag by x months
monthly["lag_1"] = monthly.groupby("SKU")["Quantity"].shift(1)
monthly["lag_2"] = monthly.groupby("SKU")["Quantity"].shift(2)
monthly["lag_3"] = monthly.groupby("SKU")["Quantity"].shift(3)

# month sin/cos seasonality
monthly["month_sin"] = np.sin(2 * np.pi * monthly["month_num"] / 12)
monthly["month_cos"] = np.cos(2 * np.pi * monthly["month_num"] / 12)

# Stratify SKUs into sales volume

In [433]:
# total volume per SKU (use your monthly/weekly aggregated frame)
sku_vol = weekly.groupby("SKU")["Quantity"].sum().sort_values(ascending=False)

# cumulative revenue/volume share
cum_share = (sku_vol.cumsum() / sku_vol.sum())

bins   = [-np.inf, 0.70, 0.95, np.inf]
labels = ["highest", "medium", "rest"]

sku_tier = pd.cut(cum_share, bins=bins, labels=labels, include_lowest=True).astype(str)
sku_tier.name = "tier"

In [434]:
print(sku_tier.value_counts())
print((sku_tier.value_counts(normalize=True) * 100).round(2).astype(str) + "%")

tier
rest       835
medium     465
highest     38
Name: count, dtype: int64
tier
rest       62.41%
medium     34.75%
highest     2.84%
Name: proportion, dtype: object


In [435]:
tiered = monthly.merge(sku_tier, left_on="SKU", right_index=True, how="inner")

# Train Test Split

In [436]:
!pip install scikit-learn

In [437]:
from sklearn.model_selection import train_test_split

In [438]:
tiered["Quantity"] = tiered["Quantity"].clip(lower=1)
tiered["lag_1"] = tiered["lag_1"].clip(lower=1)

tiered["log_qty"] = np.log1p(tiered["Quantity"])


In [439]:
X = monthly[["month_num", "year", "sku_id","lag_1", "lag_2", "lag_3", "month_sin", "month_cos"]]
y = monthly["Quantity"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


# Training and Evaluation

In [442]:
def train_xgb_core_api(df_tier, feat=("month_num", "year", "sku_id","lag_1", "lag_2", "lag_3", "month_sin", "month_cos")):
    df_tier = df_tier.dropna(subset=["lag_1"]).reset_index(drop=True)
    if len(df_tier) < 20:
        return None, None, None, None, None  # too small

    X = df_tier[list(feat)].to_numpy(dtype=float)
    y = df_tier["Quantity"].to_numpy(dtype=float)

    split_idx = int(len(df_tier) * 0.8)
    X_tr, X_va = X[:split_idx], X[split_idx:]
    y_tr, y_va = y[:split_idx], y[split_idx:]

    dtr = xgb.DMatrix(X_tr, label=y_tr, feature_names=list(feat))
    dva = xgb.DMatrix(X_va, label=y_va, feature_names=list(feat))

    params = {
        "objective": "reg:squarederror",
        "eta": 0.1,
        "max_depth": 4,
        "subsample": 0.9,
        "colsample_bytree": 0.9,
        "eval_metric": ["rmse","mae"],
        "seed": 42,
    }
    ev = [(dtr, "train"), (dva, "valid")]
    bst = xgb.train(params, dtr, num_boost_round=500, evals=ev, early_stopping_rounds=50, verbose_eval=False)

    pred = bst.predict(dva, iteration_range=(0, bst.best_iteration + 1))
    mae = float(np.mean(np.abs(y_va - pred)))
    rmse = float(np.sqrt(np.mean((y_va - pred)**2)))
    mape = float(np.mean(np.abs((y_va - pred) / np.clip(y_va, 1e-8, None))) * 100)
    smape = 100 * np.mean(2 * np.abs(y_va - pred) / (np.abs(y_va) + np.abs(pred) + 1e-8))

    return bst, mae, rmse, mape, smape, list(feat)

In [447]:
models = {}
metrics = []
future_all = []

# --- next-month forecast per tier (DROP-IN FIX) ---
for tier in ["highest", "medium", "rest"]:
    sub = tiered[tiered["tier"] == tier].copy()
    bst, mae, rmse, mape, smape, feat = train_xgb_core_api(sub)
    if bst is None:
        print(f"[{tier}] skipped (not enough data)")
        continue
    models[tier] = (bst, feat)
    print(f"[{tier}] MAE={mae:.3f}  RMSE={rmse:.3f}  MAPE={mape:.2f}%  SMAPE={smape:.2f}%  (best_iter={bst.best_iteration})")

    # last row per SKU
    sub = sub.sort_values(["SKU","Month"])
    last_rows = sub.groupby("SKU", as_index=False).tail(1).reset_index(drop=True)

    # maps for prior 2/3 months
    def nth_from_end(s, n):
        import numpy as np
        return s.iloc[-n] if len(s) >= n else np.nan

    lag2_map = sub.groupby("SKU")["Quantity"].apply(lambda s: nth_from_end(s, 2))
    lag3_map = sub.groupby("SKU")["Quantity"].apply(lambda s: nth_from_end(s, 3))

    # future date + features
    fut_month = pd.to_datetime(last_rows["Month"]) + pd.offsets.MonthEnd(1)
    fut_month_num = fut_month.dt.month

    future = pd.DataFrame({
        "SKU": last_rows["SKU"],
        "Month": fut_month,
        "month_num": fut_month_num,
        "year": fut_month.dt.year,
        "sku_id": last_rows["sku_id"],
        "lag_1": last_rows["Quantity"],                      # last observed
        "lag_2": last_rows["SKU"].map(lag2_map),             # second last
        "lag_3": last_rows["SKU"].map(lag3_map),             # third last
        "month_sin": np.sin(2 * np.pi * fut_month_num / 12),
        "month_cos": np.cos(2 * np.pi * fut_month_num / 12),
    })

    # ensure exact feature set/order
    for c in feat:
        if c not in future.columns:
            future[c] = np.nan
    future = future[list(feat)]

    dfut = xgb.DMatrix(future.to_numpy(dtype=float), feature_names=list(feat))
    preds = models[tier][0].predict(dfut, iteration_range=(0, models[tier][0].best_iteration + 1))

    out = last_rows[["SKU"]].copy()
    out["Month"] = fut_month
    out["predicted_qty"] = preds
    out["tier"] = tier
    future_all.append(out[["SKU","Month","predicted_qty","tier"]])

future_preds = pd.concat(future_all, ignore_index=True)
print(future_preds.head(20))


[highest] MAE=5.371  RMSE=6.673  MAPE=134.24%  SMAPE=55.12%  (best_iter=47)
[medium] MAE=1.435  RMSE=3.104  MAPE=77.32%  SMAPE=52.47%  (best_iter=31)
[rest] MAE=0.019  RMSE=0.022  MAPE=1.86%  SMAPE=1.83%  (best_iter=11)
             SKU                     Month  predicted_qty     tier
0   Basic/Single 2025-01-31 00:00:00+00:00      13.760201  highest
1           CH-1 2025-09-30 00:00:00+00:00       9.731747  highest
2             CU 2025-10-31 00:00:00+00:00      24.609013  highest
3           CU-1 2025-10-31 00:00:00+00:00      25.975203  highest
4           DC12 2024-02-29 00:00:00+00:00      11.686588  highest
5   DC1254/OK-WH 2023-07-31 00:00:00+00:00      30.210798  highest
6    DC1254/OKWH 2024-09-30 00:00:00+00:00       6.869424  highest
7           DR50 2025-04-30 00:00:00+00:00      17.071932  highest
8    SB4301/OKWH 2024-05-31 00:00:00+00:00       4.351573  highest
9           SC15 2025-10-31 00:00:00+00:00       7.581873  highest
10       SC15-AD 2025-10-31 00:00:00+00:00 